## Imports

In [1]:
# Imports

import pandas as pd
import numpy as np
import acquire
import prepare
import explore
from statistics import mean



import re
import unicodedata
import pandas as pd
import nltk

import prepare
import acquire

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns



## Call In DF

In [2]:
df = acquire.get_github_geology_results(cached=True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1841 entries, 0 to 1840
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   language  1841 non-null   object        
 1   content   1841 non-null   object        
 2   date      1841 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 57.5+ KB
None


,language,content,date
0,Jupyter Notebook,hylite\nhylite is an open-source python packag...,2020-11-27 17:31:07.762
1,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:08.305
2,R,"Reforester\nReforester, an R program that:\n\n...",2020-11-27 17:31:08.926
3,JavaScript,Swissgeol NGM\nA Geology 3D viewer\nSwissgeol ...,2020-11-27 17:31:09.532
4,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,2020-11-27 17:31:10.206


In [3]:
df.language.value_counts()

Python              463
Jupyter Notebook    460
R                   232
JavaScript          229
TypeScript          229
Batchfile           228
Name: language, dtype: int64

In [4]:
#Clean
df = prepare.prep_data(df, 'content')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 924 entries, 1 to 1840
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   language         924 non-null    object        
 1   content          924 non-null    object        
 2   date             924 non-null    datetime64[ns]
 3   text_cleaned     924 non-null    object        
 4   text_tokenized   924 non-null    object        
 5   text_lemmatized  924 non-null    object        
 6   text_filtered    924 non-null    object        
 7   words            924 non-null    object        
 8   doc_length       924 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 72.2+ KB
None


,language,content,date,text_cleaned,text_tokenized,text_lemmatized,text_filtered,words,doc_length
1,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:08.305,\n\n\n\n\n\nwebviz subsurface\n\n live demo ap...,webviz subsurface\n\n live demo application\n\...,webviz subsurface live demo application introd...,webviz subsurface live demo introduction conta...,"[webviz, subsurface, live, demo, introduction,...",153
2,R,"Reforester\nReforester, an R program that:\n\n...",2020-11-27 17:31:08.926,reforester\nreforester an r program that\n\nus...,reforester\nreforester an r program that\n\nus...,reforester reforester an r program that us a l...,logistic model simulated phylogenetic tree her...,"[logistic, model, simulated, phylogenetic, tre...",340
3,JavaScript,Swissgeol NGM\nA Geology 3D viewer\nSwissgeol ...,2020-11-27 17:31:09.532,swissgeol ngm\na geology d viewer\nswissgeol i...,swissgeol ngm\na geology d viewer\nswissgeol i...,swissgeol ngm a geology d viewer swissgeol is ...,cesiumjs adapt developing behaviour lava cesiu...,"[cesiumjs, adapt, developing, behaviour, lava,...",14
4,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,2020-11-27 17:31:10.206,geomodeldweb\n\ngenerates d web versions of ge...,geomodeldweb\n\ngenerates d web versions of ge...,geomodeldweb generates d web version of geolog...,geomodeldweb generates web model primarily geo...,"[geomodeldweb, generates, web, model, primaril...",114
9,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:13.263,\n\n\n\n\n\nwebviz subsurface\n\n live demo ap...,webviz subsurface\n\n live demo application\n\...,webviz subsurface live demo application introd...,webviz subsurface live demo introduction conta...,"[webviz, subsurface, live, demo, introduction,...",153


In [25]:
df.language.value_counts()

Python        463
R             232
JavaScript    229
Name: language, dtype: int64

In [26]:
# check balance of dataset for under/over sampling
explore.check_proportion(df)
# balance is not bad, will not need to over/under sample

,n,percent
Python,463,0.501082
R,232,0.251082
JavaScript,229,0.247835


In [27]:
# create a list of words for each language category and remove single letter words
jupyter_words, js_words, r_words = explore.create_lang_word_list(df)
# get the count of words by category
jupyter_freq, js_freq, r_freq = explore.get_count_word_freq(jupyter_words, js_words, r_words)

AttributeError: 'DataFrame' object has no attribute 'lemmatized'

## Ave Length of Readme

In [6]:
# python_doc_length = round(mean(df[df.language == 'Python'].doc_length), 2)
# jupyter_doc_length = round(mean(df[df.language == 'Jupyter Notebook'].doc_length),2)
# r_doc_length = round(mean(df[df.language == 'R'].doc_length),2)
# javascript_doc_length = round(mean(df[df.language == 'JavaScript'].doc_length),2)
# typescript_doc_length = round(mean(df[df.language == 'TypeScript'].doc_length),2)
# batchfile_doc_length = round(mean(df[df.language == 'Batchfile'].doc_length),2)

# print(f'The average length of the python repositories is {python_doc_length} words')
# print(f'The average length of the jupyter repositories is {jupyter_doc_length} words')
# print(f'The average length of the r repositories is {r_doc_length} words')
# print(f'The average length of the javascript repositories is {javascript_doc_length} words')
# print(f'The average length of the typescript repositories is {typescript_doc_length} words')
# print(f'The average length of the batchfile repositories is {batchfile_doc_length} words')

## Explore List of Words

In [7]:
# # Create list of words
# #python_words = ' '.join(df[df.language=='Python'].text_filtered)
# jupyter_words = ' '.join(df[df.language=='Jupyter Notebook'].text_filtered)
# r_words = ' '.join(df[df.language=='R'].text_filtered)
# #typescript_words = ' '.join(df[df.language=='TypeScript'].text_filtered)
# javascript_words = ' '.join(df[df.language=='JavaScript'].text_filtered)
# #batchfile_words = ' '.join(df[df.language=='Batchfile'].text_filtered)
# all_words = ' '.join(df.text_filtered)

# # remove spaces
# #python_words = re.sub(r'\s.\s', '', python_words)
# jupyter_words = re.sub(r'\s.\s', '', jupyter_words)
# r_words = re.sub(r'\s.\s', '', r_words)
# #typescript_words = re.sub(r'\s.\s', '', typescript_words)
# javascript_words = re.sub(r'\s.\s', '', javascript_words)
# #batchfile_words = re.sub(r'\s.\s', '', batchfile_words)
# all_words = re.sub(r'\s.\s', '', all_words)

In [8]:
# # wordcounts

# #python_freq = pd.Series(python_words.split()).value_counts()
# jupyter_freq = pd.Series(jupyter_words.split()).value_counts()
# r_freq = pd.Series(r_words.split()).value_counts()
# #typescript_freq = pd.Series(typescript_words.split()).value_counts()
# javascript_freq = pd.Series(javascript_words.split()).value_counts()
# #batchfile_freq = pd.Series(batchfile_words.split()).value_counts()
# all_freq = pd.Series(all_words.split()).value_counts()

## What repositories detail work in Geology

In [9]:
# No geology keywords in top 20 most frequent words
# python_freq.head(20)

In [10]:
# Several geology keywords in to 20
# jupyter_freq.tail(20)

In [11]:
# many geology keywords in top 20
# r_freq.tail(20)

In [12]:
# No geology Content
#typescript_freq.head(20)

In [13]:
# many geology references
# javascript_freq.tail(20) 

In [14]:
#***

#`Removed Word List`
#1. Went back and edited clean to remove numbers
#2. Went back and identified most frequent words for each language
#   and removed all words that were not explicitly related to geology content

["accuracy","accurate","acknowledgement","acquired",
 "adhere","adheres","along","also","accessing","provided",
 "directory", "datasets","including","pip","problem","pull",
 "python",
 "according","account","across","addition","additional","aim",
 "anaconda","analyse","analysed","analysis","andor","answer",
 "according","public","proxy","properly","prompt","program",
 "produce", "produced","reference","need","optimization","viewer",
 "open","norequestrendermode","preform","performed","point","pointcloud",
 "question",
 "install","data","import","using","please","different","installation",
 "preprocessing","library","type","implemented","minimum","github",
 "run","using","file","license","result","record","possible","number",
 "could","see","use","source","define","parameter","geology","disable",
 "limit","inspector","quality","default","display", "activate","alternative",
 "anacona","associated","assumption","attempt","author","automated","automatic",
 "available","purpose","published","publically","prevented","different","navigation",
 "quality",'nolimit',"package",
 "al","application","assetids","based","better","bias","biased","birthdeath","raw",
 "rather","present","prerequisitewindowv",
 "particular","partially","oxford","requires","happy","request","report","high",
 "provide","project","processing","potential","calculated","cannot","capture","carried",
 "cause",'cc','cd',
 "python","conda","variety","notebook","work","image","sample","check","running","console",
 "example","easily","map","sampling","true","method","version","script","many","set","url",
 "ngm","npm",'swissgeol','swisstopo','list', 'behavior','git','licencemd','restrict','swiss',
 'mind',"initialscreenspaceerror",
 "gdal","following","welcomed","terminal","workflow","cloud","review","jupyter","laboratory",
 "technique","navigate","test","time","used","pde","reforester","database","past","research",
 "section","visual","developingmd","ownterrainfalse","widget","json","cpu",'new','welcome','id',
 "integrated","opening","learning","code","marker","corrected","trying","citing","though",'refer',
 "history",'gnu',"specie",'dataimporterr','dissertation','wa','httpsbetaswissgeolch','layout',
 'software','swissrectangle','localhost','except','start','httplocalhost','keyboard','disables',
 'us',
 'machine','server','documentation','testing','integration','cool','create','stay','due','study',
 'estimate','pattern','general','error','enforce','made','started','local','clone','modify','sphere',
 'coma','getting','nolimitfalse','rectangle','maximumscreenspaceerror',
 "io","fix","userhylite","dimensionality","ioloadtestdatahypercloudplycloudquickplotcloudheadergetcamera",
 "cite","hylitergb","specific","thiele","lib","pdes","included","occurrence","recent","must","significant",
 "texture","httpsopengameartorgcontenttemplateorangetexturepack","swissrectanglefalse","note",
 "launch", "conventionally","mark","technology","fast","opencv","marking","take","efficiently","sure","make",
 "solved","unsupervised","know","long","matplotlib","numpy","created","imagine",'prerequisitewindowv','longheld',
 "word","phytools","placed","eg","indicate","complete","received","term","results","format","desired",
 "templatelogistic",'template','r',"keyboardlayouteditor",
 "httpsgithubcomswissgeolngmgit","dev","dtilesets",
 "touchdirectly","step","issue","correction","highresolution","find","derivative","geological","want",
 "scan","supervised","ioloadtestdatalibrarycsvlibquickplot",'information',"level","exponentially","optional",
 "implied","google","thing","idea","said","timeheterogeneous", "entierty","inferred","tidyverse","detailed",
 "roger","still", "estimation","appearance",
 "next","ground","opensource","typing","preform","corescannerhypercloud","one","design","et","httpshylitereadthedocsioenlatestindexhtml",
 "openpit",'installed','scene','templatenotebooks','difficult','setuppy','significantly','benson','even',
 "referred","compared","lineagecombining","inverse","daniel","regime","vast","function","functionsuniformsamplingr",
 "factor","fine","equilibrium","length",'myrs','myr','empirical',
 "doe","includes","launching","try","ioloadtestdatalibrarycsvlibquickplot","contributing","touchdirectly",
 "generate","multiscale","userhylite","simple","ioloadtestdatahypercloudplycloudquickplotcloudheadergetcamera",
 "feature","perform","lorenz","respectively","merchantability","order","end","least","marineeumetazoacsv",
 "window","ubiquity","second","completeness","theoretically","originatorclose","hope","common","operation",
 "similarly","value","inaccurate","thereby",
 "ioloadtestdatalibrarycsvlibquickplot", "tuned","currently","touchdirectly","unzip","ioloadtestdataimagehdrimagequickplothylitergb",
 "submit","setuptools","repository","corescannerhypercloud","seamless","commented","prerequisitev","exponentiallydiversifying",
 "httpswwwgnuorglicenses","consistently","ha","viewed","suggests","slight","introduces","closer","maximum",
 "unfortunately","billie","readme","caper","restriction"
]





#***

['accuracy',
 'accurate',
 'acknowledgement',
 'acquired',
 'adhere',
 'adheres',
 'along',
 'also',
 'accessing',
 'provided',
 'directory',
 'datasets',
 'including',
 'pip',
 'problem',
 'pull',
 'python',
 'according',
 'account',
 'across',
 'addition',
 'additional',
 'aim',
 'anaconda',
 'analyse',
 'analysed',
 'analysis',
 'andor',
 'answer',
 'according',
 'public',
 'proxy',
 'properly',
 'prompt',
 'program',
 'produce',
 'produced',
 'reference',
 'need',
 'optimization',
 'viewer',
 'open',
 'norequestrendermode',
 'preform',
 'performed',
 'point',
 'pointcloud',
 'question',
 'install',
 'data',
 'import',
 'using',
 'please',
 'different',
 'installation',
 'preprocessing',
 'library',
 'type',
 'implemented',
 'minimum',
 'github',
 'run',
 'using',
 'file',
 'license',
 'result',
 'record',
 'possible',
 'number',
 'could',
 'see',
 'use',
 'source',
 'define',
 'parameter',
 'geology',
 'disable',
 'limit',
 'inspector',
 'quality',
 'default',
 'display',
 'activat

In [15]:
# All in spannish, not so helpful to me
#batchfile_freq.head(20)

In [16]:
#all_freq.head(20)

***
### Focus on Repositories That have Geology Content
 
 - `No`: Python, Typescrpt, Batchfile   
 
 - `Yes`: Jupyter, R, Javascript   

***

In [17]:
# Dataframe to compare most frequent words

# word_counts = (pd.concat([all_freq, python_freq, jupyter_freq, r_freq, typescript_freq, javascript_freq, batchfile_freq], axis=1, sort=True)
#                .set_axis(['all', 'python', 'jupyter', 'r', 'typescript', 'javascript', 'batchfile'], axis=1, inplace=False)
#                .fillna(0)
#                .apply(lambda s: s.astype(int))
#               )

# word_counts.head(20)

# word_counts = (pd.concat([jupyter_freq, r_freq, javascript_freq], axis=1, sort=True)
#                .set_axis(['jupyter', 'r', 'javascript'], axis=1, inplace=False)
#                .fillna(0)
#                .apply(lambda s: s.astype(int))
#               )

# word_counts.head(50)

In [18]:
# These might be words to excude to make modeling easier
# word_counts[word_counts.jupyter == word_counts.jupyter].sort_values(by='jupyter').head(10)

In [19]:
#These might be words to excude to make modeling easier
# word_counts[word_counts.r == word_counts.r].sort_values(by='r').head(10)

In [20]:
#These might be words to excude to make modeling easier
# word_counts[word_counts.javascript == word_counts.javascript].sort_values(by='javascript').head(10)

In [21]:
# visualize the percentage of the term in jupyter, r, javascript
# (word_counts.assign(p_python = word_counts.python/word_counts['all'], 
#                    p_jupyter = word_counts.jupyter/word_counts['all'],
#                    p_r = word_counts.r/word_counts['all'],
#                    p_typescript = word_counts.typescript/word_counts['all'],
#                    p_javascript = word_counts.javascript/word_counts['all'],
#                    p_batchfile = word_counts.batchfile/word_counts['all'])
#  .sort_values(by='all')[['p_python', 'p_jupyter', 'p_r','p_typescript','p_javascript','p_batchfile']]
#  .tail(20)
#  .sort_values('p_jupyter')
#  .plot.barh(stacked=True, figsize = (20,10))
# )

# plt.title("Proportion of 20 most common keywords in GitHub Geology README Files", fontsize = 20)
# plt.show()


In [22]:
# combine list of word counts into df for further exploration
# word_counts = explore.create_df_word_counts(jupyter_freq, r_freq, javascript_freq)
# word_counts.head()

***
`Next, compute proportion of each string that for each language.
Because the languages are relatively balanced, we should see about the same proportion as the distibution in the dataset. If we see significantly higher or lower proportion, then that word might be a good feature in predicting that language.
A chi-square test could confirm or deny significance if needed.`
***

In [23]:
# word_counts = explore.word_counts_proportion(word_counts)
# word_counts.head(10)

In [24]:
# visualize the % of the term in each language
explore.proportion_visualization(word_counts)

NameError: name 'word_counts' is not defined

In [ ]:
# # Wordcloud jupyter_notebook
# plt.figure(figsize=(10,10))
# text = jupyter_words
# wc = WordCloud(background_color="white",
#                max_words=2000, max_font_size=224,
#                random_state=42)
# wc.generate(text)
# plt.imshow(wc, interpolation="bilinear")
# plt.axis('off')
# plt.show()

In [ ]:
# # Wordcloud js
# plt.figure(figsize=(10,10))
# text = javascript_words
# wc = WordCloud(background_color="white",
#                max_words=2000, max_font_size=224,
#                random_state=42)
# wc.generate(text)
# plt.imshow(wc, interpolation="bilinear")
# plt.axis('off')
# plt.show()

In [ ]:
# # Wordcloud R
# plt.figure(figsize=(10,10))
# text = r_words
# wc = WordCloud(background_color="white",
#                max_words=2000, max_font_size=224,
#                random_state=42)
# wc.generate(text)
# plt.imshow(wc, interpolation="bilinear")
# plt.axis('off')
# plt.show()

## Bigrams and Trigrams

In [ ]:
# #Create list of bigrams
# ham_bigrams = pd.Series(list(nltk.ngrams(ham_words.split(), 2))).value_counts().hea

In [ ]:
# addd code